## Potential Energy Surface Python Code
<br>
This script allows the user to enter in a diatomic molecule to compute the Potential Energy Surface through of the Hartree-Fock method. 
<br> <br>
The program is limited to only using Hydrogen and Helium atoms due to the Hartree-Fock base code only supporting s-orbitals. 
<br><br>
The potential energy surface graph uses Hartrees as units for the energy, and bohr units for representing the diatomic bond distance. 

In [1]:
from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from morsePotential import morsePotential
from notebookImporter import importNotebook
from IPython.display import clear_output

HartreeFock = importNotebook("Hartree_Class")

#set up the automated calculations used to generate the surface potential

#User Specified Variables

#Enter the atomic number for atoms 1 and 2
atom1 = 1
atom2 = 1

#Enter the number of electrons for atom 1 and 2
#Note the total number of electrons must be a multiple of 2, 
#otherwise, the computation will round the total number of electrons 
#down to the nearest multiple of 2
atomN1 = 1
atomN2 = 1

#Select a basis, for a list of available basis sets, please see the "basisSets" folder
basisName = "STO-3G"

#Enter Starting and End Bond distance for the diatomic molecule
#Units for Bond Distance in This Program are Bohr Units
startR = .85
endR = 5

#Please enter the distance the bond distance should increase each iteration
#The smaller the number, the more accurate and time-consuming the computation will be
step = 0.01

#Please enter how small the difference in energy between two ground state energy calculations
#must be in order to convergence to occur
convergenceCritera = pow(10, -10)

#Please enter the maximum number of iterations the SCF procedure can take before
#the computation will fail as being unable to converge
maxIterations = 100

#System Defined Variables
E = []
R = []
basisSets = []
Xs = []
MOEnergy = []
delta = (endR - startR) / step
currentR = startR

#Main While Loop to repeatedly call the Hartree-Fock routine to generate the potential energy surface
while(not(currentR >= endR)):
    
    #define the molecular system
    system = molecule()
    system.addAtom(atom(vector(1,1,2+currentR), atom1, atomN1))
    system.addAtom(atom(vector(1,1,2), atom2, atomN2))

    system.addBasis(basisName)
    
    #Set up the Hartree Procedure
    HF = HartreeFock.HF(system, convergenceCritera, maxIterations)

    #Store Bond distance, associated Energy, and MO Energies
    R.append(currentR) 
    E.append(HF.SCF() )
    MOEnergy.append(HF.MOEnergy)
    
    print("Hartree-Fock Computation: " + str(currentR * 100 // endR) + "%")
    
    #update the current intenuclear distance
    currentR += step
    
print("Hartree-Fock Computation Complete")

#E.append(0)
#R.append(12)

Hartree-Fock Computation: 17.0%
Hartree-Fock Computation: 17.0%
Hartree-Fock Computation: 17.0%
Hartree-Fock Computation: 17.0%
Hartree-Fock Computation: 17.0%
Hartree-Fock Computation: 18.0%
Hartree-Fock Computation: 18.0%
Hartree-Fock Computation: 18.0%
Hartree-Fock Computation: 18.0%
Hartree-Fock Computation: 18.0%
Hartree-Fock Computation: 19.0%
Hartree-Fock Computation: 19.0%
Hartree-Fock Computation: 19.0%
Hartree-Fock Computation: 19.0%
Hartree-Fock Computation: 19.0%
Hartree-Fock Computation: 20.0%
Hartree-Fock Computation: 20.0%
Hartree-Fock Computation: 20.0%
Hartree-Fock Computation: 20.0%
Hartree-Fock Computation: 20.0%
Hartree-Fock Computation: 21.0%
Hartree-Fock Computation: 21.0%
Hartree-Fock Computation: 21.0%
Hartree-Fock Computation: 21.0%
Hartree-Fock Computation: 21.0%
Hartree-Fock Computation: 22.0%
Hartree-Fock Computation: 22.0%
Hartree-Fock Computation: 22.0%
Hartree-Fock Computation: 22.0%
Hartree-Fock Computation: 22.0%
Hartree-Fock Computation: 23.0%
Hartree-

In [2]:
#Setup system for graphing purposes
import math
import scipy.integrate as integrate
import scipy.optimize as optimize
import numpy as np
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


#create a curve fitted morsePotential of the data
#uses the least squares method to fit the potential via SciPy
def morsePotential(r, r0, a, D, c):
    return c+ D*pow( (1-np.exp(-(a*(r-r0)))), 2) 
    
    #return D*pow(1 - np.exp(-a*(r-r0)), 2) + c
    #return D * np.exp(-2*a*(r-r0)) - 2*D*np.exp(-a*(r-r0)) + c 
    #return a0 * (a1 * (r - r0) + a2 * pow((r - r0), 2) + a3 * pow(r-r0, 3)) + a4
    #return (a1 * np.exp(-2 * a2 * ( r - r0)) - (2 * a1 * np.exp(-a2 * ( r - r0)))) + c
    #return (a1 * np.exp(-a2 * r) * (1 - (a3*r))) - (a4 / (pow(r,6) + (a5*pow(r,-6)))) 
    #return A - (D*( 1 + a1*(r-r0) + a2*pow((r-r0),2) + a3*pow(r-r0, 3) ) * np.exp(-a1*(r-r0)))
    #return c - D*( 1 + a*(r-r0)  ) * np.exp(-a*(r-r0))

    
popt, pcov = optimize.curve_fit(morsePotential, R, E)

#Prepare data for graphing
minEIndex = E.index(min(E))
minE = E.pop(minEIndex)
optimalR = R.copy().pop(minEIndex)

In [8]:
from scipy.misc import derivative as ddx

#u refers to the reduced mass of the diatomic molecule in Hartree Atomic Units
#The mass of a proton is 1836 a.u.
u = (1836*1836)/(1836 + 1836)

#w = 1 / ((1 / (2 * math.pi) ) * math.sqrt( 2 * popt[2] * pow(popt[1], 1) ))
#w = popt[1] / ( math.pi * math.sqrt( 2 * u1 / popt[2] ) ) 
#w = math.sqrt( 2 * pow(popt[1], 2) * popt[2] / u1 ) 
w = 0.005
#w = 0.020052
# w = 0.02

print(w)
print("*"*20)

re = 1
unitConverter = 1
mc = 0 #-popt[3]

def r0f(n):
    return 3

#Returns a lambda function of the nth hermite polynomial
def hermite(n, k):
    
    c = pow(-1, k) * math.factorial(n) / ( math.factorial(k) * math.factorial(n - 2*k) )
        
    if(k == 0):
        return lambda r : c * pow(2*math.sqrt(u*w)*( (r-r0f(n)) / unitConverter ), n-(2*k))
    else:
        return lambda r : c * pow(2*math.sqrt(u*w)*( (r-r0f(n)) / unitConverter ), n-(2*k)) + hermite(n, k-1)(r)

#Constructs the nth hermite polynomial function, and returns the specified lambda function
def buildHermite(n):
    return hermite(n, n // 2)

#returns the normalization constant for the Harmonic Oscillator Function
def C(n):    
    return math.sqrt( math.sqrt(u*w) /  (pow(2, n) * math.factorial(n) * np.sqrt(math.pi * pow(unitConverter,2)) ) ) 

#Creates a new Harmonic Oscillator function
#and returns it as a lambda function
def newHO(n):
    return lambda r : C(n) * buildHermite(n)(r) * np.exp(-u * w * pow((r-r0f(0)) / unitConverter,2) / 2)  

basisSize = 20

#Set up all variables used for the integral computation
basisSet = []
S = np.zeros([basisSize, basisSize])
V = np.zeros([basisSize, basisSize])
T = np.zeros([basisSize, basisSize])
T2 = np.zeros([basisSize, basisSize])

Tints = []
Vints = []

#Build the basis Fucntions
for i in range(basisSize):
    basisSet.append( newHO(i) )
    
#Verify orthonormality of the basis set
for i in range(basisSize):
    for j in range(basisSize):
        
#         integrand = lambda r : basisSet[i](r) * basisSet[j](r)
        
#         S[i,j] += round(integrate.quad(integrand, 0, 50, limit=10000, epsabs=pow(10, -50), epsrel=pow(10, -30))[0],4)
        
#         if( (i == j and abs(S[i, j] - 1) >= .1 or not 1==j and abs(S[i,j]) >= pow(10, -4) )):
#             print("Overlap Error: i=" + str(i) + ", j=" + str(j))
#             print(S[i,j])
            
        integrand = lambda r : basisSet[i](r) * (morsePotential(r, popt[0], popt[1], popt[2], popt[3] + mc)) * basisSet[j](r)
        
        V[i, j] += integrate.quad( integrand, 0, np.inf)[0]#, limit=10000, epsabs=pow(10, -50), epsrel=pow(10, -30))[0]
 
        integrand2 = lambda r : basisSet[i](r) * (-1/(2*u)) * ddx(basisSet[j], r, n=2)

        T[i, j] += integrate.quad(integrand2, 0, np.inf)[0]
    
        Tints.append(integrand2)
        Vints.append(integrand)
        
        if (i==j):
            T2[i,j] += 2*i + 1
        elif (i == j + 2):
            T2[i,j] += -math.sqrt(i * (i - 1)) 
        elif (i == j - 2):
            T2[i, j] += -math.sqrt( (i+1) * (i+2) )
   
T2 *= w / 4

print()
print("Overlap Matrix: ")
print()
print(S)
print()
print("*" * 40)
print()

print("V")
print()
print(V)
print()
print("*"*40)
print()

print("T")
print()
print(T)
print()
print("*" * 40)
print()

H = T + V

print(H)
print()

EHO = sorted(np.linalg.eig(H)[0])
print(EHO)

print(len(basisSet))

print("!!"*10)
print(T2)



0.005
********************

Overlap Matrix: 

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0

In [9]:
#Finite Difference Method from Schrier

#def morsePotential(r):
#    D =  -0.17455
#    b = .02 #8.794 * pow(10, -18)
#    r0 = 1.4
#    return D * pow( 1 - math.exp( -b * (r - r0) ), 2 )
#    return 0

#popt[0] = 1.4
#popt[1] = 8.794 * pow(10, -18)
#popt[2] = 0.17455
#popt[3] = 0

points = 1000

#Start and end in Bohr Radius
start = 0
end = 300

#mass of the molecule in Hartree Atomic Units
m = (1836 * 1836) / (2 * 1836)

dr = (end - start) / points

H = np.zeros([points, points])

for i in range(start, points):
    for j in range(start, points):
        
        if(i == j):
            h = 1 / ( m * pow(dr, 2)) + morsePotential(dr * j, popt[0], popt[1], popt[2], popt[3])
        elif(i == j-1 or i == j+1):
            h = - 1 / ( 2 * m * pow(dr, 2) )
        else:
            continue
        
        H[i, j] += h
        
print(H)

print()
print("EIG VAL " * 5)
print()

#EHO = sorted(np.linalg.eigh(H)[0])

print(EHO)


[[ 0.30447907 -0.0060518   0.         ...  0.          0.
   0.        ]
 [-0.0060518  -0.42364476 -0.0060518  ...  0.          0.
   0.        ]
 [ 0.         -0.0060518  -0.82041557 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... -0.60315664 -0.0060518
   0.        ]
 [ 0.          0.          0.         ... -0.0060518  -0.60315664
  -0.0060518 ]
 [ 0.          0.          0.         ...  0.         -0.0060518
  -0.60315664]]

EIG VAL EIG VAL EIG VAL EIG VAL EIG VAL 

[-1.1089733338344823, -1.0958165361296672, -1.076400462545418, -1.0523463762687728, -1.0207218988045454, -0.9857485632570632, -0.9538594362413723, -0.9258839574775524, -0.895770611957539, -0.8641789212900148, -0.8337695023765025, -0.8055423596655065, -0.77974677013559, -0.7563592735213429, -0.7352652390476794, -0.7163185700603483, -0.6992730706160021, -0.6832118221885786, -0.6680712098607091, -0.5761689608347349]


In [ ]:
#prepare data for use with graphing
dx = .01
steps = 15/dx

data = [x*dx for x in range(int(steps))]

figure = { 
    "data": [
        
        #create the Hartree-Fock generated Potential Energy Surface
        {
            "type":"scatter",
            "x":R,
            "y":E,
            "connectgaps":False,
            "mode":"markers", 
            "name":"Hartree-Fock Computed",
            "marker":{"color":"blue"}
        },
    
        #Highlight the minimum energy point in red
        {
            "type":"scatter",
            "x":[optimalR],
            "y":[minE],
            "name":"Optimal Bond Distance",
            "marker":{"color":"red"}
        },
        
        #Create and plot the Morse Potential fit
        {
            "type":"scatter",
            "x":[x*dx for x in range(0, 1000)],
            "y": [morsePotential(r * dx, popt[0], popt[1], popt[2], popt[3]) for r in range(0, 1000)], #popt[0], popt[1], popt[2], popt[3]) for r in data] ,
            "connectgaps":False,
            "name":"Morse Potential Approximation",
            "marker":{"color":"green"}
        },
        
 #       {
 #           "type":"scatter",
 #           "x":[popt[0]] * len(EHO),
 #           "y":[x for x in EHO],
 #           "connectgaps":False,
 #           "name":"Harmonic Oscillator",
 #           "marker":{"color":"blue"}
 #       },
        #{
        #    "type":"scatter",
        #    "x":[popt[1]] * len(EPW),
        #    "y":EPW,
        #    "connectgaps":False,
        #    "name":"Plane Wave",
        #    "marker":{"color":"blue"}
        #},
        
           #{
           # "type":"scatter",
           # "x":[PIB] * len(EPW),
           # "y":PIB,
           # "connectgaps":False,
           # "name":"PIB Solution",
           # "marker":{"color":"blue"}
        #}
        
    ],
    
    #Set up the layout of the graph
    "layout":
        {
           "xaxis":{"title":"Bond Distance in Bohr Units Radius"},
           "yaxis":{"title":"Energy in Hartrees"},
            "title":{"text":"Hartree-Fock Energy VS Bond Distance"}
        },    
}

xMin = 0
xMax = 10
steps = 500
dx = (xMax - xMin) / steps

#Loop over all MO Energies
#for i, bf in enumerate(basisSet):
#        figure["data"].append(
#            {
#                "type":"scatter",

#                "x":[x*dx for x in range(0, 1000)],
#                "y" : [bf(x*dx) for x in range(0, 1000) ],
#                "name":"HO " + str(i)
#            }
#        )    
#Loop over all MO Energies
for i in range(basisSize):
    for j in range(basisSize):
        figure["data"].append(
            {
                "type":"scatter",
                "x":[x*dx for x in range(0, 500)],
                "y" : [basisSet[i](x*dx) * morsePotential(x*dx, popt[0], popt[1], popt[2], popt[3]) * basisSet[j](x*dx) for x in range(0, 500) ],
                #"y":[basisSet[i](r) * (1/(2*u)) * ddx(basisSet[j], r, n=2) for r in range(0, 100)],

                "name":"VINT " + str(i) + " " + str(j)
            }
        )    
        
for i in range(len(EHO)):

    figure["data"].append(
        {
            "type":"scatter",
            "x":[popt[0]],
            "y":[EHO[i]],
            "connectgaps":False,
            "name":"T" + str(i),
            "marker":{"color":"blue"}
        }
    )    

iplot(figure)

#Testing to find out units of the basis set
#Omega as bohr to centimeter FAIL

#Omega as bohr to meter FAIL

In [6]:
#